In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [4]:
df = pd.read_csv(r"C:\Users\manah\Downloads\monatszahlen2505_verkehrsunfaelle_06_06_25.csv")
df.head()

,MONATSZAHL,AUSPRAEGUNG,JAHR,MONAT,WERT,VORJAHRESWERT,VERAEND_VORMONAT_PROZENT,VERAEND_VORJAHRESMONAT_PROZENT,ZWOELF_MONATE_MITTELWERT
0,Alkoholunfälle,insgesamt,2025,202501,NaN,NaN,NaN,NaN,NaN
1,Alkoholunfälle,insgesamt,2025,202502,NaN,NaN,NaN,NaN,NaN
2,Alkoholunfälle,insgesamt,2025,202503,NaN,NaN,NaN,NaN,NaN
3,Alkoholunfälle,insgesamt,2025,202504,NaN,NaN,NaN,NaN,NaN
4,Alkoholunfälle,insgesamt,2025,202505,NaN,NaN,NaN,NaN,NaN


In [ ]:
    """
    Understanding of the dataset
    MONATSZAHL: CATEGORY
    AUSPRAEGUNG:ACCIDENT TYPE
    JAHR: YEAR
    MONAT: MONTH
    WERT: VALUE
    VORJAHRESWERT: PREVIOUS YEAR COUNT
    VERAEND_VORMONAT_PROZENT: PERCENTAGE CHANGE COMPARED TO PREVIOUS MONTH
    VERAEND_VORJAHRESMONAT_PROZENT: PERCENTAGE CHANGE TO THE SAME MONTH IN PREVIOUS YEAR
    ZWOELF_MONATE_MITTELWERT: AVERAGE OVER 12 MONTHS
    """
